In [1]:
# Basic imports
import os
import sys
import cv2 
import numpy as np  
import pandas as pd 
import openslide

sys.path.append(os.path.abspath('..'))
from utils.utils_data import get_veesel_sheets, get_segmentations
from utils.utils_geometry import get_contours_by_classification, is_contour_match_bounds

from utils.utils_vis import gallery_view
from utils.utils_constants import (ARTERY_TYPES,
                                   CLASSIFICATION_PATH,
                                   SEGMENTATION_DIR,
                                   TRI_CASE_DIR,
                                   CROPPED_VESSELS_DIR)
import matplotlib.pyplot as plt

import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [2]:
SLIDE_FILENAME = "11_26609_000_009_L2_TRI.svs"

In [3]:
def vis_slide(classifications, segmentations):
    for artery_type in ARTERY_TYPES:
        print(artery_type)
        images = []
        titles = []
        for _, row in classifications[classifications["Artery Type"] == artery_type].iterrows():
            bbox_x, bbox_y, bbox_width, bbox_height = map(int, row["Bounding Box"].split(","))

            outer_contours = get_contours_by_classification(
                segmentations,
                lambda contour: is_contour_match_bounds(
                    contour, (bbox_x, bbox_y, bbox_width, bbox_height)
                ),
                "Media"
            )
            assert(len(outer_contours)) == 1
            # vessel_size = int(np.log(cv2.contourArea(outer_contours[0])))
            area = cv2.contourArea(outer_contours[0])
            vessel_size = f"{round(area):.1e}"

            # cv2.drawContours(slide, outer_contours, -1, [255, 0, 0], 2)  # Outer contour in red
            # Construct the path to the image file
            image_path = os.path.join(CROPPED_VESSELS_DIR, artery_type, 
                                    row["Image Name"].replace(".png", "_w_ann.png"))
            img = cv2.imread(image_path, cv2.IMREAD_COLOR)  # Correct function to load the image
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for correct color display
            images.append(img)
            # Create a title using multiple fields from the DataFrame
            title = f"Size: {vessel_size}; AS: {row['Arteriosclerosis Severity']}; HS: {row['Hyalinosis Severity']}"
            # title = f"B{row['Image Name'].split('_')[1]}{row['Artery ID']}-AS: {row['Arteriosclerosis Severity']}; HS: {row['Hyalinosis Severity']}"
            titles.append(title)
        if len(images) > 0:
            gallery_view(images, titles, cols=4)

In [4]:
def get_border_of_cnt(cnt, border=0):
    # cnt: coordinates, list of list, [[0, 0], [0, 1], ...]
    cnt = np.array(cnt, dtype=int).squeeze()
    xmin, xmax = np.min(cnt[:, 0]), np.max(cnt[:, 0])
    ymin, ymax = np.min(cnt[:, 1]), np.max(cnt[:, 1])
    return (xmin-border, xmax+border, ymin-border, ymax+border)

def get_border_of_ann(ann, border=50):
    # ann: need to be cleaned to get coordinates
    (xmin, xmax) = (float('inf'), 0)
    (ymin, ymax) = (float('inf'), 0)
    for (i, ann_i) in enumerate(ann):
        coords = ann_i['geometry']['coordinates']
        curr_xmin, curr_xmax, curr_ymin, curr_ymax = get_border_of_cnt(coords)
        xmin = min(xmin, curr_xmin)
        xmax = max(xmax, curr_xmax)
        ymin = min(ymin, curr_ymin)
        ymax = max(ymax, curr_ymax)

    return (xmin-border, xmax+border, ymin-border, ymax+border)

In [ ]:
available_sheetnames = pd.ExcelFile(CLASSIFICATION_PATH).sheet_names
slide_path = os.path.join(TRI_CASE_DIR, SLIDE_FILENAME)
slide_0 = openslide.OpenSlide(slide_path)
slide_basename = os.path.splitext(SLIDE_FILENAME)[0]
classifications = get_veesel_sheets(CLASSIFICATION_PATH, slide_basename, available_sheetnames, remove_others=True)
segmentations_path = os.path.join(SEGMENTATION_DIR, f"{slide_basename}.geojson")
segmentations = get_segmentations(segmentations_path, clean=True)
xmin, xmax, ymin, ymax = get_border_of_ann(segmentations, border=0)
slide = slide_0.read_region((xmin, ymin), 0, (xmax-xmin, ymax-ymin))

plt.imshow(np.array(slide))
plt.axis("off")
plt.show()

vis_slide(classifications, segmentations)